# Задание 4
- Шеверев Сергей Вячеславович
- 22М-05ММ
- Вариант 3: зависимая переменная $\texttt{irritability.1}$, остальные переменные будем считать независимыми.
    Независимые переменные упорядочим по значимости влияния на зависимую переменную, приведем соответсвующие частоты, значимости критерия хи-квадрат и точного критерия Фишера.
    
    По совместному распределению первых двух наиболее значимых переменных вычислим односторонний коэффициент неопределенности относительно зависимой переменной.
    

In [289]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats

In [290]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

Считаем датасет:

In [291]:
df = pd.read_csv('data_big.csv')

In [292]:
len(df.keys())

121

В датасете 120 признаков, один из которых - это порядковый номер записи. Нам нужны наблюдения в первый день и только  качественные признаки. Поэтому отфильтруем ненужные.

In [293]:
df.head()

,Unnamed: 0,X.1,depressed.mood.1,anxiety.1,suspiciousness.1,irritability.1,craving.to.alcohol.1,weakness.1,insomia.1,headache.1,...,combined2.9,HR.9,SBP.9,DBP.9,MBP.9,SV.9,CO.9,SI.9,CI.9,TPR.9
0,1,1,1,1,0,1,1,1,1,0,...,0,68.0,108.0,70.0,83.0,74.0,5.0,40.0,2.7,1321.0
1,2,2,1,1,0,0,1,1,2,1,...,9,63.0,114.0,70.0,85.0,123.0,7.7,60.0,3.8,879.0
2,3,3,1,1,0,0,0,2,1,0,...,0,64.0,120.0,80.0,93.0,106.0,6.8,55.0,3.5,1098.0
3,4,4,2,2,0,0,0,2,0,0,...,0,56.0,124.0,90.0,101.0,90.0,5.1,43.0,2.4,1600.0
4,5,5,1,1,0,0,2,2,1,0,...,0,66.0,116.0,78.0,90.0,90.0,5.9,45.0,2.2,1228.0


In [294]:
df = df [df.keys()[2:20]]

In [295]:
df.isnull().sum()

depressed.mood.1              0
anxiety.1                     0
suspiciousness.1              0
irritability.1                0
craving.to.alcohol.1          0
weakness.1                    0
insomia.1                     0
headache.1                    0
tremor.1                      0
polyuria.1                    0
sweating.1                    0
transient.hallusinations.1    0
vomiting.1                    0
anoreksia.1                   0
diarrhea.1                    0
thirst.1                      0
chest.pain.1                  0
hyperemia.1                   0
dtype: int64

Пропущенных значений нет - хорошо

Определим вспомогательные функции:
 - `entropy` - функция для подсчета энтропии
 - `conditional_probs` - функция для подсчета условных вероятностей (возвращает список)
 - `chisquare` - функция теста хи-квадрат
 - `uncertain` - функция для вычисления коэффициентов неопределенности

In [296]:
def entropy(feature =  None, counts =  None):
    e = 0.0
    if feature is not None:
        tmp = {}
        for i in feature:
            tmp[i] = tmp.get(i, 0) + 1
        for i in tmp.keys(): 
            e += float(tmp[i])/len(feature)*np.log2(float(tmp[i])/len(feature))
    else: 
        probs = counts/counts.sum()
        for i in probs:
            if i == 0:
                continue
            e += i*np.log2(i)        
    return -e




def conditional_probs(X, Y):

    
    crosstab = pd.crosstab(X, Y)
    row_sums = sum(np.array(crosstab).T)
    conditionals = crosstab.iloc[:, 1:].sum(axis=1)/row_sums
    crosstab['Сумма'] = row_sums
    crosstab['Условные вероятности'] = conditionals
    
    
    #ct = np.array(pd.crosstab(X, Y), dtype='float64')
    #row_sums = sum(ct.T)
    #for i in range(ct.shape[0]):
    #    ct[i] /= row_sums[i]
    return crosstab


#хи-квадрат 
def chisquare(X, Y):
    ct = np.array(pd.crosstab(X, Y))
    N = sum(sum(ct))#сначала сумма по столбцам, потом суммируем суммы по столбцам
    col_sums = sum(ct)
    row_sums = sum(ct.T)
    chi2 = 0
    for i in range(ct.shape[0]):
        for k in range(ct.shape[1]):
            chi2 += np.power(ct[i][k],  2)/row_sums[i]/col_sums[k]
    chi2 = (chi2 -1)*np.sum(ct)
    df = (ct.shape[0] - 1)*(ct.shape[1] - 1)
    return 1 - scipy.stats.chi2.cdf(df = df, x=chi2)





def uncertain(X, Y = None , tab = None):
    ct = None
    if tab is None:
        ct = np.array(pd.crosstab(X, Y))
    else:
        ct = np.array(tab)
    XY_entropy = entropy(counts = ct.flatten())
    X_entropy = entropy(counts = sum(ct.T))
    Y_entropy = entropy(counts = sum(ct))
    information = X_entropy + Y_entropy - XY_entropy
    return min(information/X_entropy*100, 100.0), min(information/Y_entropy*100, 100.0), min(2*information/(Y_entropy+X_entropy)*100, 100.0)

Посчитаем
 - энтропию каждого признака
 - таблицу условных вероятностей
 - точный критерий Пирсона
 - точный критерий Фишера
 - коэффициенты неопределенности

In [335]:
X = df['irritability.1']
a = lambda x: 0 if x < 1 else 1
X_ = X.apply(a) if len(X.unique()) > 2 else X
uncertains = []
chi2s = []
fishers = []
keys = []
print(f'Энтропия признака irritability.1 {entropy(X_)}\n\n')
for i in df.keys():
    if i == 'irritability.1':
        continue
    keys.append(i)
    Y = df[i]
    #a = lambda x: 1 if x < 2 else 2
    a = lambda x: 0 if x < 1 else 1
    Y_ = Y.apply(a) if len(Y.unique()) > 2 else Y
    print(f'Признак {i}:')
    print('Таблица сопряженности с условными вероятностями:\n',conditional_probs(X_, Y_))

    chi2s.append(chisquare(X_, Y_))
    print(f'Точный критерий Пирсона: {chi2s[-1]}')
    
    
    fisher_tab = pd.crosstab(X_, Y_)
    if fisher_tab.shape == (2, 2):
        fp = scipy.stats.fisher_exact(fisher_tab)[1]
        fishers.append(fp)
        print(f'Точный критерий Фишера:  {fp}' + (' > 0.05 => различия можно объянить случайностью' if fp > 0.05 else ' < 0.05 => различия нельзя объяснить случайностью'))
    print(f'Энтропия {i}: {entropy(Y_)}')
    uncertains.append(uncertain(X_, Y_))
    
    print(f'Коэффициенты неопределенности: {uncertains[-1]}')
    
    print('\n\n')

Энтропия признака irritability.1 0.9596868937742169


Признак depressed.mood.1:
Таблица сопряженности с условными вероятностями:
 depressed.mood.1   1  2  Сумма  Условные вероятности
irritability.1                                      
0                 17  4     21              0.190476
1                 11  2     13              0.153846
Точный критерий Пирсона: 0.7854126368894419
Точный критерий Фишера:  1.0 > 0.05 => различия можно объянить случайностью
Энтропия depressed.mood.1: 0.6722948170756379
Коэффициенты неопределенности: (0.16628484892415316, 0.23736816957757603, 0.19556762074574163)



Признак anxiety.1:
Таблица сопряженности с условными вероятностями:
 anxiety.1       0   1  Сумма  Условные вероятности
irritability.1                                    
0               4  17     21              0.809524
1               1  12     13              0.923077
Точный критерий Пирсона: 0.3636020765912179
Точный критерий Фишера:  0.6271778506123857 > 0.05 => различия можно объянить

Коэффициенты неопределенности: (14.049433971329545, 13.483057647231211, 13.76042028965536)



Признак hyperemia.1:
Таблица сопряженности с условными вероятностями:
 hyperemia.1     0   1  Сумма  Условные вероятности
irritability.1                                    
0               1  20     21              0.952381
1               0  13     13              1.000000
Точный критерий Пирсона: 0.4245061631338082
Точный критерий Фишера:  1.0 > 0.05 => различия можно объянить случайностью
Энтропия hyperemia.1: 0.19143325481419343
Коэффициенты неопределенности: (2.1717459519500597, 10.88732534332459, 3.6211617514461536)





Два самых влиятельных признака:

In [337]:
t= sorted(zip(keys, chi2s, fishers, uncertains), key=lambda x: x[1], reverse=False)

pd.DataFrame(data = {'Признак': [i[0] for i in t],\
                     'Критерий Пирсона': [i[1] for i in t],\
                     'Критерий Фишера': [i[2] for i in t],\
                     'Коэффициенты неопределенности': [i[3] for i in t]}
            )
#for i in t[:2]:
#    print(i)


,Признак,Критерий Пирсона,Критерий Фишера,Коэффициенты неопределенности
0,chest.pain.1,0.013499,0.032361,"(14.049433971329545, 13.483057647231211, 13.76..."
1,craving.to.alcohol.1,0.055960,0.074963,"(8.670368695849909, 8.88345143920585, 8.775616..."
2,vomiting.1,0.106312,0.210646,"(5.6424644426177775, 6.879451500044623, 6.1998..."
3,headache.1,0.140547,0.167588,"(4.779026740039108, 4.779026740039108, 4.77902..."
4,weakness.1,0.153526,0.270053,"(6.784183045062712, 15.121735735393017, 9.3662..."
5,transient.hallusinations.1,0.153526,0.270053,"(6.784183045062712, 15.121735735393017, 9.3662..."
6,anoreksia.1,0.153526,0.270053,"(6.784183045062712, 15.121735735393017, 9.3662..."
7,polyuria.1,0.248028,0.386772,"(2.877071686555787, 3.764069832614534, 3.26133..."
8,diarrhea.1,0.251406,0.513369,"(4.430653032175988, 13.174122288071185, 6.6311..."
9,anxiety.1,0.363602,0.627178,"(1.9757772630870993, 3.1474611490308266, 2.427..."



Посмотрим на соотвествующие относительные частоты:

In [338]:
crosstab = pd.crosstab(df['irritability.1'].apply(a), [ df[t[0][0]].apply(a) , df[t[1][0]].apply(a)  ] )
crosstab

chest.pain.1          0     1   
craving.to.alcohol.1  0  1  0  1
irritability.1                  
0                     7  7  3  4
1                     1  2  1  9

Посмотрим на совместное распределение:

In [339]:
uncertain(df['irritability.1'], tab = crosstab)

(20.170470997165456, 10.229449691989153, 13.57456293646457)

Наличие признаков $ \texttt{chest.pain.1} \text{ и } \texttt{craving.to.alcohol.1}$ определеяет почти 20% информации о зависимом признаке $ \texttt{irritability.1}$ 
